# Loop 1 Analysis: Ensemble Strategy

## Goal
1. Score all available snapshots per-N
2. Find the best solution for each N value
3. Create an ensemble submission
4. Identify which N values have the most room for improvement

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm

# Tree vertices
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

def score_group(xs, ys, degs):
    """Calculate score for a single N-tree configuration"""
    n = len(xs)
    all_x, all_y = [], []
    for i in range(n):
        rad = np.radians(degs[i])
        c, s = np.cos(rad), np.sin(rad)
        for j in range(len(TX)):
            x = TX[j] * c - TY[j] * s + xs[i]
            y = TX[j] * s + TY[j] * c + ys[i]
            all_x.append(x)
            all_y.append(y)
    side = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
    return side * side / n

def parse_submission(df):
    """Parse submission and return per-N scores"""
    df = df.copy()
    # Parse the 's' prefix from values
    df['x_val'] = df['x'].astype(str).str.replace('s', '', regex=False).astype(float)
    df['y_val'] = df['y'].astype(str).str.replace('s', '', regex=False).astype(float)
    df['deg_val'] = df['deg'].astype(str).str.replace('s', '', regex=False).astype(float)
    
    # Extract N from id (e.g., '003_1' -> 3)
    df['n'] = df['id'].str.split('_').str[0].astype(int)
    
    per_n_scores = {}
    per_n_data = {}
    
    for n in range(1, 201):
        group = df[df['n'] == n]
        if len(group) == n:
            xs = group['x_val'].values
            ys = group['y_val'].values
            degs = group['deg_val'].values
            score = score_group(xs, ys, degs)
            per_n_scores[n] = score
            per_n_data[n] = group[['id', 'x', 'y', 'deg']].copy()
    
    return per_n_scores, per_n_data

print('Functions defined')

In [ ]:
# Find all submission files in snapshots
snapshot_dir = '/home/nonroot/snapshots/santa-2025'
snapshot_dirs = sorted(glob.glob(f'{snapshot_dir}/*/'))
print(f'Found {len(snapshot_dirs)} snapshot directories')

# Collect all submission files
submission_files = []
for d in snapshot_dirs:
    sub_file = os.path.join(d, 'submission', 'submission.csv')
    if os.path.exists(sub_file):
        submission_files.append(sub_file)

print(f'Found {len(submission_files)} submission files')

In [ ]:
# Score all submissions and track best per-N
best_per_n = {n: {'score': float('inf'), 'data': None, 'source': None} for n in range(1, 201)}

for sub_file in tqdm(submission_files, desc='Scoring submissions'):
    try:
        df = pd.read_csv(sub_file)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            continue
        per_n_scores, per_n_data = parse_submission(df)
        
        for n, score in per_n_scores.items():
            if score < best_per_n[n]['score']:
                best_per_n[n]['score'] = score
                best_per_n[n]['data'] = per_n_data[n]
                best_per_n[n]['source'] = sub_file
    except Exception as e:
        print(f'Error processing {sub_file}: {e}')

print('\nDone scoring all submissions')

In [ ]:
# Calculate total ensemble score
total_ensemble_score = sum(best_per_n[n]['score'] for n in range(1, 201) if best_per_n[n]['data'] is not None)
print(f'Total ensemble score from snapshots: {total_ensemble_score:.6f}')

# Compare to baseline
baseline_score = 70.625918
print(f'Baseline score: {baseline_score:.6f}')
print(f'Improvement: {baseline_score - total_ensemble_score:.6f}')

# Show score breakdown by range
print('\nScore breakdown by N range:')
for start, end in [(1, 50), (51, 100), (101, 150), (151, 200)]:
    range_score = sum(best_per_n[n]['score'] for n in range(start, end+1))
    print(f'  N={start}-{end}: {range_score:.4f}')

In [ ]:
# Show which N values improved vs baseline
baseline_df = pd.read_csv('/home/code/experiments/001_baseline/baseline.csv')
baseline_per_n, _ = parse_submission(baseline_df)

improvements = []
for n in range(1, 201):
    baseline_n = baseline_per_n.get(n, float('inf'))
    ensemble_n = best_per_n[n]['score']
    if ensemble_n < baseline_n - 1e-9:
        improvements.append((n, baseline_n, ensemble_n, baseline_n - ensemble_n))

print(f'\nN values with improvements: {len(improvements)}')
if improvements:
    improvements.sort(key=lambda x: x[3], reverse=True)
    print('\nTop 20 improvements:')
    for n, old, new, diff in improvements[:20]:
        print(f'  N={n}: {old:.6f} -> {new:.6f} (improvement: {diff:.6f})')

In [ ]:
# Create ensemble submission
ensemble_rows = []
for n in range(1, 201):
    if best_per_n[n]['data'] is not None:
        ensemble_rows.append(best_per_n[n]['data'])

ensemble_df = pd.concat(ensemble_rows, ignore_index=True)

# Sort by N and index
ensemble_df['sn'] = ensemble_df['id'].str.split('_').str[0].astype(int)
ensemble_df['si'] = ensemble_df['id'].str.split('_').str[1].astype(int)
ensemble_df = ensemble_df.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
ensemble_df = ensemble_df[['id', 'x', 'y', 'deg']]

print(f'Ensemble submission shape: {ensemble_df.shape}')
print(ensemble_df.head())

In [ ]:
# Save ensemble submission
os.makedirs('/home/code/experiments/002_ensemble_snapshots', exist_ok=True)
ensemble_df.to_csv('/home/code/experiments/002_ensemble_snapshots/submission.csv', index=False)
ensemble_df.to_csv('/home/submission/submission.csv', index=False)

print(f'Saved ensemble submission with score: {total_ensemble_score:.6f}')

In [ ]:
# Also check research kernels for additional solutions
kernel_dir = '/home/code/research/kernels'
kernel_submissions = []

for root, dirs, files in os.walk(kernel_dir):
    for f in files:
        if f.endswith('.csv'):
            kernel_submissions.append(os.path.join(root, f))

print(f'Found {len(kernel_submissions)} CSV files in research kernels')
for f in kernel_submissions:
    print(f'  {f}')

In [ ]:
# Score kernel submissions and update best_per_n
for sub_file in kernel_submissions:
    try:
        df = pd.read_csv(sub_file)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            continue
        per_n_scores, per_n_data = parse_submission(df)
        
        for n, score in per_n_scores.items():
            if score < best_per_n[n]['score']:
                best_per_n[n]['score'] = score
                best_per_n[n]['data'] = per_n_data[n]
                best_per_n[n]['source'] = sub_file
                print(f'Found better N={n}: {score:.6f} from {sub_file}')
    except Exception as e:
        pass

# Recalculate total
total_with_kernels = sum(best_per_n[n]['score'] for n in range(1, 201) if best_per_n[n]['data'] is not None)
print(f'\nTotal score with kernels: {total_with_kernels:.6f}')